### Streaming 

Methods : stream() and astream()
- these methods are sync and async methods for straming back results. 

Additional parameters in streaming modes for graph state. 

- **values** : this streams the full state of the graph after each node is called.
- **update** : this streams updates to the state of the graph after each node is called.

![](../images/img3.png)